In [ ]:
%matplotlib inline

import re
import numpy as np
import subprocess
import os, stat
import os.path
import time
import tempfile
import shutil
import logging
import urllib
import enum

import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
logging.getLogger("").setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logging.getLogger("").addHandler(ch)

logger = logging.getLogger("main")
logger.debug("Test")

In [ ]:
!python --version

In [ ]:
class Architecture(enum.Enum):
    LAPTOP = 1
    DESKTOP = 2
    SUPERCOMPUTER = 3
    
architecture = Architecture.SUPERCOMPUTER


In [ ]:
#git_url = "git@github.com:babrodtk/gpu-ocean.git"
git_url = None
git_command = "git"
my_env = os.environ
shell = True

# If we are on windows, we might need to modify the path and git command:
if os.name == 'nt':
    my_env["PATH"] = "C:\\Program Files\\Git\\mingw64\\bin\\;C:\\Program Files\\Git\\usr\\bin\\;" + my_env["PATH"]
    git_command = "git.exe"
if os.name == 'posix':
    shell = False

try:
    git_url = subprocess.check_output([git_command, "rev-parse", "--show-toplevel"], stderr=subprocess.STDOUT, shell=shell)
    git_url = os.path.abspath(os.path.join(git_url.decode("utf-8").rstrip(), ".git"))
except subprocess.CalledProcessError as e:
    logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
logger.info(git_url)
    
    
benchmark_script_relpath = "gpu_ocean/demos/testCasesDemos/run_benchmark.py"
cdklm_python_class_relpath = "gpu_ocean/SWESimulators/CDKLM16.py"
benchmark_script_version = "5c2214573269367fbdb6d67ace07f2ffa57d37ba"
benchmark_script_options = ["--simulator", "CDKLM", '--steps_per_download', '10', '--iterations', '3']
outfile = os.path.join(os.getcwd(), "output_" + time.strftime("%Y_%m_%d-%H_%M_%S") + ".npz")


git_versions = [
    #(git hash, git commit message, block size laptop, block size desktop, block size supercomputer)
    ("a126bab97e690b7c642814e3c8b96f9879adf487", "original (per arch optimized block size)", (32,4), (12,12), (12, 12)), #Change blocksize here!
    ("5d817bb7cd2b369039117d19aae7d669a9a2e53a", "Optimized shared memory use Hm", (32,4), (12,12), (16, 16)),
    ("2e5da01457874ad5087398da77b9102ee991fb94", "Optimized shared memory use RHx/RHy", (32,4), (12,12), (16, 16)),
    ("fcd145c8c29f3d38a024685fdd0fc3cf9580366c", "Optimized shared memory use Q", (32,4), (12,12), (16, 16)),
    ("74e957bb41e391a5c5fbb19c3cac393079272dd3", "Optimized shared memory use F", (32,4), (12,12), (16, 16)),
    ("5aba525d1d64937c03e8d2b33bb7f6d80b97a81a", "Optimized shared memory use G", (32,4), (12,12), (16, 16)),
    ("addb061fe1cfccebb6fab70f9826be3752162b0b", "Optimized shared memory use Qy", (32,4), (12,12), (16, 16)),
    ("aab115045fd6d313940fa37be3149155ba4ead70", "Made variables const", (32,4), (12,12), (16, 16)),
    ("f6a911368b429df608eb5e1218c28bc14d6ffbe2", "Updated blocksize experiment code", (32,4), (12,12), (16, 16)),
    ("339eb0c9a3bf04875507fab24d8924e24718a2eb", "Changed default block size", (16,8), (24,8), (8, 12)), #Change blocksize here!
    ("1319edf92c20b8d5fd9fc8c0e0ed58176f9cc2ba", "Tried to reduce register use", (16,8), (24,8), (16, 16)),
    ("6de871265b820aecc63d68a2e1f858605eb367c7", "Refactored to optimize register use", (16,8), (24,8), (16, 16)),
    ("f6d1727d747b486203df91079b83a9e717668852", "Added logger to CDKLM", (16,8), (24,8), (16, 16)),
    ("40f9d4817fa084de7cebfd3cf6d20254b93e7393", "Added fast math compilation flag", (16,8), (24,8), (16, 16)),
    #("e2af2159becbd3c8769903eb4bea602a96d6c3a1", "Compilation flags experimentation", (16,8), (24,8), (16, 16)),
    ("12536844bdc4459dcf4cc92776faea3a81d0a32c", "New optimal block size", (32,8), (32,12), (16, 8)) #Change blocksize here!
]


for version, log, laptop, desktop, supercomputer in git_versions:
    logger.info(version + " - " + log)

In [ ]:
#Create temporary directory
tmpdir = tempfile.mkdtemp(prefix='git_tmp')
logger.debug("Writing to " + tmpdir)

In [ ]:
def git(cwd, options):
    a = None
    try:
        cmd = [git_command] + options
        logger.debug("Git cloning: " + cwd + " -- " + str(cmd))
        a = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=shell, cwd=cwd, env=my_env)
    except subprocess.CalledProcessError as e:
        logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
    return a.decode("utf-8") 

    
git_clone = os.path.join(tmpdir, "git_clone")
stdout = git(tmpdir, ["clone", git_url, git_clone])
logger.debug(stdout)

In [ ]:
def block_string(block_size):
    return "block_width="+str(block_size[0])+", block_height="+str(block_size[1])

def patch_block_sizes(cdklm_py_file_path, old_block_sizes, new_block_sizes):
    old_string = block_string(old_block_sizes)
    new_string = block_string(new_block_sizes)
    
    # create temporary file
    tmpfile, tmpfile_abs_path = tempfile.mkstemp()
    with os.fdopen(tmpfile, 'w') as new_file:
        with open(cdklm_py_file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(old_string, new_string))
    # Remove original file
    os.remove(cdklm_py_file_path)
    # Move the new file
    shutil.move(tmpfile_abs_path, cdklm_py_file_path)


In [ ]:
for version, log, laptop_block, desktop_block, supercomputer_block in git_versions:
    logger.debug("checkout " + version)
    logger.debug("    with commit msg: " + log)
    stdout = git(git_clone, ["checkout", "--force", '-q', version])
    logger.debug("stdout: " + str(stdout))
    
    logger.debug("Checkout " + benchmark_script_relpath)
    stdout = git(git_clone, ["checkout", "--force", '-q', benchmark_script_version, "--", benchmark_script_relpath])
    logger.debug("stdout: " + str(stdout))
        
    if architecture == Architecture.DESKTOP or architecture == Architecture.SUPERCOMPUTER:
        cdklm_abs_path = os.path.join(git_clone, cdklm_python_class_relpath)
        if architecture == Architecture.DESKTOP:
            patch_block_sizes(cdklm_abs_path, laptop_block, desktop_block)
            logger.debug("Changed blocksizes from " + str(laptop_block) + " to " + str(desktop_block))
    
        if architecture == Architecture.SUPERCOMPUTER:
            patch_block_sizes(cdklm_abs_path, laptop_block, supercomputer_block)
            logger.debug("Changed blocksizes from " + str(laptop_block) + " to " + str(desktop_block))
        
        # Grep on file to check that we succeeded:
        try:
            cmd = ["grep", "block_width=", cdklm_abs_path]
            a = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=shell, cwd=tmpdir, env=my_env)
        except subprocess.CalledProcessError as e:
            logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
        logger.debug("Confirming block sizes through grep:\n" + a.decode("utf-8"))
        
    a = None
    try:
        #cmd = ["python", "--version"]
        cmd = ["python", os.path.join(git_clone, benchmark_script_relpath)] + benchmark_script_options + ["--output", "benchmark_" + version + ".npz"]
        a = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=shell, cwd=tmpdir, env=my_env)
    except subprocess.CalledProcessError as e:
        logger.error("Failed, return code " + str(e.returncode) + "\n" + str(e.output))
    logger.debug("Output:\n" + a.decode("utf-8"))
    
    # Cleaning git folder so that it can be deleted
    #stdout = git(git_clone, ["-n", ])
        
print("benchmarking finished")

In [ ]:
print("hei")

In [ ]:
versions, labels, laptop_block, desktop_block, supercomputer_block = list(zip(*git_versions))
megacells = np.full((len(versions)), np.nan)

for i, ver in enumerate(git_versions):
    version, log, l_block, d_block, hpc_block  = ver
    filename = os.path.join(tmpdir, "benchmark_" + version + ".npz")
    with np.load(filename) as version_data:
        megacells[i] = version_data['CDKLM']

np.savez(outfile, versions=versions, labels=labels, megacells=megacells)

## Plot results

In [ ]:
### NOTE: Static files
outfile_laptop = os.path.join(os.getcwd(), "laptop_cdklm_profiling.npz")
outfile_desktop = "desktop_cdklm_profiling.npz"
outfile_supercomputer = "supercomputer_fermi_cdklm_profiling.npz"
###

with np.load(outfile_laptop) as data:
    logger.debug("laptop file: " + outfile_laptop)
    versions = data['versions']
    labels = data['labels']
    megacells_laptop = data['megacells']
    normalized_megacells_laptop = megacells_laptop / megacells_laptop[0]
    
with np.load(outfile_desktop) as data:
    assert(np.all(versions == data['versions']))
    logger.debug("desktop file: " + outfile_desktop)
    megacells_desktop = data['megacells']
    normalized_megacells_desktop = megacells_desktop / megacells_desktop[0]
    
with np.load(outfile_supercomputer) as data:
    assert(np.all(versions == data['versions']))
    logger.debug("supercomputer file: " + outfile_supercomputer)
    megacells_supercomputer = data['megacells'] 
    normalized_megacells_supercomputer = megacells_supercomputer / megacells_supercomputer[0]
    
fig = plt.figure()

fig, ax = plt.subplots(figsize=(12,8))

index = np.arange(len(versions))
bar_width = 0.25

ax.bar(index, normalized_megacells_laptop, bar_width, label='Laptop')
ax.bar(index+bar_width, normalized_megacells_desktop, bar_width, label='Desktop')
ax.bar(index+2*bar_width, normalized_megacells_supercomputer, bar_width, label='Supercomputer')

plt.xticks(index, labels, rotation='vertical')
plt.grid()

plt.legend()

In [ ]:
def remove_readonly(func, path, _):
    "Clear the readonly bit and reattempt the removal"
    os.chmod(path, stat.S_IWRITE)
    func(path)

shutil.rmtree(tmpdir, onerror=remove_readonly)
logger.debug("Removed tempdir " + tmpdir)